# Desarrollo librería de manejo de datos

In [3]:
import json
from pandas.core.base import DataError
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import timedelta
import numpy as np
from os import path
import matplotlib.pyplot as plt

import sys
from pathlib import Path
sys.path.insert(1, '/utils/')
sys.path.insert(1, '../src2/utils/')
from timeutils import timeJStoPy 

In [4]:
def dataretriever(user=None,password=None):
    # Esta funcion devolvera una funcion de request        
    def request(endpoint):
        if user:
            aux = requests.get('https://api.cv19gm.org/'+endpoint, auth=HTTPBasicAuth(user, password))
            if aux.status_code == 401:
                raise Exception('Wrong Credentials')
            else:
                print('Logged in successfully')
        else:
            aux = requests.get('http://192.168.2.223:5006/'+endpoint)
        return aux
    return request


def dfappend(df,values,days,dataname):
    length = len(df)
    aux = [np.nan for i in range(length)]
    iterator = iter(values)
    for i in days:
        aux[i] = next(iterator)
    auxdf = pd.DataFrame({dataname:aux})
    return pd.concat([df,auxdf],axis=1)

In [7]:
request = dataretriever()

# Ocupación Hospitalaria

## Nacional

In [41]:
endpoint = 'getAllBedsAndVentilation'

In [43]:
data = request(endpoint).json()

In [45]:
data.keys()

dict_keys(['dates', 'camas_ocupadas', 'camas_totales', 'vmi_covid19_confirmados', 'vmi_covid19_sospechosos', 'vmi_ocupados', 'vmi_totales'])

In [44]:
pd.DataFrame(data)

,dates,camas_ocupadas,camas_totales,vmi_covid19_confirmados,vmi_covid19_sospechosos,vmi_ocupados,vmi_totales
0,2020-04-12T04:00:00.000Z,1827.0,2851,318,39,720,1635
1,2020-04-13T04:00:00.000Z,1798.0,2877,335,39,685,1623
2,2020-04-15T04:00:00.000Z,1955.0,3002,314,32,692,1683
3,2020-04-16T04:00:00.000Z,1979.0,2978,316,29,706,1668
4,2020-04-18T04:00:00.000Z,1898.0,2964,352,29,718,1658
...,...,...,...,...,...,...,...
178,2020-10-26T03:00:00.000Z,3341.0,4211,604,17,1123,2550
179,2020-10-27T03:00:00.000Z,3371.0,4214,596,14,1115,2530
180,2020-10-28T03:00:00.000Z,3357.0,4219,604,17,1123,2535
181,2020-10-29T03:00:00.000Z,3308.0,4208,602,20,1130,2530


## Regional

In [19]:
endpoint1 = 'getRegionalIcuBedOccupation'
endpoint2 = 'getBedsAndVentilationByState?state='

In [18]:
state = '13'

In [22]:
data1 = request(endpoint1).json()
data2 = request(endpoint2+state).json()

In [47]:
data1['data'].keys()

dict_keys(['10', '11', '12', '13', '14', '15', '16', '01', '02', '03', '04', '05', '06', '07', '08', '09'])

In [30]:
data1['data']['13'].keys()

dict_keys(['capacity', 'occupied_covid', 'occupied_non_covid'])

In [48]:
data2.keys()

dict_keys(['dates', 'camas_ocupadas', 'camas_totales', 'vmi_covid19_confirmados', 'vmi_covid19_sospechosos', 'vmi_ocupados', 'vmi_totales'])

In [29]:
pd.DataFrame(data2)

,dates,camas_ocupadas,camas_totales,vmi_covid19_confirmados,vmi_covid19_sospechosos,vmi_ocupados,vmi_totales
0,2020-04-12T04:00:00.000Z,1005.0,1564,156,22,345,789
1,2020-04-13T04:00:00.000Z,991.0,1571,169,27,321,792
2,2020-04-15T04:00:00.000Z,1111.0,1649,156,20,338,838
3,2020-04-16T04:00:00.000Z,1117.0,1646,157,15,339,840
4,2020-04-18T04:00:00.000Z,1076.0,1617,187,15,354,812
...,...,...,...,...,...,...,...
178,2020-10-26T03:00:00.000Z,1873.0,2328,288,8,576,1304
179,2020-10-27T03:00:00.000Z,1920.0,2336,289,9,590,1289
180,2020-10-28T03:00:00.000Z,1912.0,2337,293,12,601,1290
181,2020-10-29T03:00:00.000Z,1861.0,2337,285,13,600,1290


In [31]:
pd.DataFrame(data1['data']['13'])

,capacity,occupied_covid,occupied_non_covid
0,793,198,298
1,808,196,314
2,808,202,317
3,808,181,340
4,818,186,341
...,...,...,...
339,1784,1126,561
340,1802,1146,549
341,1828,1158,590
342,1831,1181,580


## Comunal

In [32]:
endpoint = 'getBedsAndVentilationByComuna?comuna='

In [33]:
comuna = '13101'

In [35]:
data = request(endpoint+comuna).json()

In [38]:
pd.DataFrame(data)

,dates,camas_ocupadas,camas_totales,vmi_covid19_confirmados,vmi_covid19_sospechosos,vmi_ocupados,vmi_totales
0,2020-04-12T04:00:00.000Z,220.0,268,33,5,83,173
1,2020-04-13T04:00:00.000Z,229.0,268,34,8,81,173
2,2020-04-15T04:00:00.000Z,232.0,268,37,7,88,173
3,2020-04-16T04:00:00.000Z,220.0,268,36,4,81,173
4,2020-04-18T04:00:00.000Z,229.0,268,42,5,91,174
...,...,...,...,...,...,...,...
178,2020-10-26T03:00:00.000Z,272.0,294,27,2,83,215
179,2020-10-27T03:00:00.000Z,276.0,302,24,2,82,215
180,2020-10-28T03:00:00.000Z,279.0,302,22,7,87,215
181,2020-10-29T03:00:00.000Z,275.0,302,25,7,97,215


In [51]:
pd.DataFrame(request('getHealthcareCenters').json())

,id_minsal,nombre_minsal,id_sochimi,nombre_sochimi,tipo_recinto,tipo_dependencia,servicio_salud,seremi,region_id,region,comuna_id,comuna,servicio_salud_id
0,101100,Hospital Regional Dr. Juan Noé Crevanni (Arica),2,Hospital Regional Dr. Juan Noé,Establecimiento Alta Complejidad,Servicio de Salud,SS ARICA,Servicio de Salud Arica,15,Arica y Parinacota,15101,Arica,151
1,101213,Clínica San José,1,Clinica San José,Clínica,Privado,SS ARICA,SEREMI De Arica y Parinacota,15,Arica y Parinacota,15101,Arica,151
2,102100,Hospital Dr. Ernesto Torres Galdames (Iquique),3,Hospital regional de Iquique,Establecimiento Alta Complejidad,Servicio de Salud,SS IQUIQUE,Servicio de Salud Iquique,01,Tarapacá,01101,Iquique,011
3,103100,Hospital Dr. Leonardo Guzmán (Antofagasta),4,Hospital regional de Antofagasta,Establecimiento Alta Complejidad,Servicio de Salud,SS ANTOFAGASTA,Servicio de Salud Antofagasta,02,Antofagasta,02101,Antofagasta,021
4,103101,Hospital Dr. Carlos Cisternas (Calama),9,Hospital del Calama,Establecimiento Alta Complejidad,Servicio de Salud,SS ANTOFAGASTA,Servicio de Salud Antofagasta,02,Antofagasta,02201,Calama,021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,900001,Clínica Red Salud Rancagua,35,Clinica Red Salud Rancagua,SIN DATO,SIN DATO,SS LIBERTADOR B. O'HIGGINS,Servicio de Salud Del Libertador B.O'Higgins,06,Lib. Gral. Bernardo O'Higgins,06101,Rancagua,061
121,900004,Hospital Clínico Regional Dr. Guillermo Grant ...,47,Gran Benavente 2,Establecimiento Alta Complejidad,Servicio de Salud,SS CONCEPCIÓN,Servicio de Salud Concepción,08,Biobío,08101,Concepción,083
122,900006,UPC Padre las Casas,122,UPC Padre las Casas,SIN DATO,SIN DATO,SS ARAUCANÍA SUR,Servicio de Salud Araucanía Sur,09,La Araucanía,09112,Padre Las Casas,092
123,900007,Hospital Metropolitano,124,Hospital Metropolitano,Hospital,Servicio de Salud,SS METROPOLITANO OCCIDENTE,Servicio de Salud Metropolitano Occidente,13,Metropolitana de Santiago,13123,Providencia,133


In [53]:
pd.DataFrame(request('getBedsAndVentilationByHealthService?health_service=151').json())

,dates,camas_ocupadas,camas_totales,vmi_covid19_confirmados,vmi_covid19_sospechosos,vmi_ocupados,vmi_totales
0,2020-04-12T04:00:00.000Z,7.0,26,5,0,6,20
1,2020-04-13T04:00:00.000Z,8.0,18,5,0,5,15
2,2020-04-15T04:00:00.000Z,8.0,18,5,2,8,15
3,2020-04-16T04:00:00.000Z,14.0,24,6,2,9,15
4,2020-04-18T04:00:00.000Z,8.0,24,5,2,8,15
...,...,...,...,...,...,...,...
178,2020-10-26T03:00:00.000Z,81.0,112,14,0,19,57
179,2020-10-27T03:00:00.000Z,82.0,112,14,0,19,57
180,2020-10-28T03:00:00.000Z,73.0,112,11,0,12,57
181,2020-10-29T03:00:00.000Z,72.0,112,11,0,12,56


In [57]:
request('getDetailedBedsAndVentilationByHealthService').json()['data']['101'].keys()

dict_keys(['dates', 'camas_ocupadas_intensivo', 'camas_ocupadas_intermedio', 'camas_totales_intensivo', 'camas_totales_intermedio', 'vmi_covid19_confirmados', 'vmi_covid19_sospechosos', 'vmi_ocupados', 'vmi_totales', 'name'])

In [58]:
request('getBedsAndVentilationByComuna?comuna=13101').json().keys()

dict_keys(['dates', 'camas_ocupadas', 'camas_totales', 'vmi_covid19_confirmados', 'vmi_covid19_sospechosos', 'vmi_ocupados', 'vmi_totales'])

# Subreporte

In [28]:
import matplotlib.dates as mdates


In [5]:
endpoint = 'getNationalNewCasesUnderreport'

In [10]:
subreporte = pd.DataFrame(request(endpoint).json())

In [15]:
dates = pd.DataFrame({'dates':[timeJStoPy(subreporte['date'][i]) for i in range(len(subreporte['date']))]})

In [20]:
subreporte.keys()

Index(['dates', 'date', 'mean', 'low', 'high'], dtype='object')

In [19]:
subreporte = pd.concat([dates,subreporte],axis=1)

In [23]:
subreporte[[ 'mean', 'low', 'high']]

,mean,low,high
0,1.008490,1.154032,0.855560
1,0.979597,1.118069,0.835338
2,0.952467,1.087118,0.812929
3,0.926488,1.058615,0.791414
4,0.902429,1.033651,0.769611
...,...,...,...
352,0.259509,0.401436,0.106292
353,0.240129,0.388088,0.080872
354,0.218263,0.367020,0.056297
355,0.196420,0.358218,0.023580


In [33]:
#4k
plt.rcParams["figure.figsize"] = 38.40, 20.56 #19.20,10.80#8,6
# Small Screen
#plt.rcParams["figure.figsize"] = 19.2, 10.80 #19.20,10.80#8,6
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif":["Arial"]})


In [46]:
import matplotlib
%matplotlib tk

In [55]:
plt.plot(subreporte['dates'],subreporte['mean'],label = 'mean')
plt.plot(subreporte['dates'],subreporte['low'],label = 'low')
plt.plot(subreporte['dates'],subreporte['high'],label = 'high')
plt.title('Subreporte',size=22)
plt.tick_params(labelsize=18)
plt.legend(loc=0,fontsize=18)
plt.fmt_xdata = mdates.DateFormatter('%d-%m-%Y')

# Paso a paso

In [59]:
endpoint = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto74/paso_a_paso.csv'

In [61]:
cuarentenas = pd.read_csv(endpoint)

In [62]:
cuarentenas

,codigo_region,region_residencia,codigo_comuna,comuna_residencia,zona,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,...,2021-03-20,2021-03-21,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29
0,15,Arica y Parinacota,15101,Arica,Rural,3,3,3,3,3,...,2,2,2,2,2,2,2,2,2,2
1,15,Arica y Parinacota,15101,Arica,Urbana,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
2,15,Arica y Parinacota,15102,Camarones,Total,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,15,Arica y Parinacota,15202,General Lagos,Total,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,15,Arica y Parinacota,15201,Putre,Total,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,12,Magallanes,12101,Punta Arenas,Total,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
377,12,Magallanes,12103,Río Verde,Total,3,3,3,3,3,...,4,4,4,4,4,4,4,4,4,4
378,12,Magallanes,12104,San Gregorio,Total,3,3,3,3,3,...,4,4,4,4,4,4,4,4,4,4
379,12,Magallanes,12303,Timaukel,Total,3,3,3,3,3,...,4,4,4,4,4,4,4,4,4,4


In [86]:
def timetxttoDatetime(t):
    return datetime.strptime(t,'%Y-%m-%d')

In [94]:
cuarentenas[cuarentenas['codigo_region']==13]

,codigo_region,region_residencia,codigo_comuna,comuna_residencia,zona,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,...,2021-03-20,2021-03-21,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29
87,13,Metropolitana,13502,Alhué,Total,3,3,3,3,3,...,2,2,2,2,2,2,2,1,1,1
88,13,Metropolitana,13402,Buin,Total,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
89,13,Metropolitana,13403,Calera de Tango,Total,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
90,13,Metropolitana,13102,Cerrillos,Total,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
91,13,Metropolitana,13103,Cerro Navia,Total,1,1,1,1,1,...,2,2,2,2,2,1,1,1,1,1
92,13,Metropolitana,13301,Colina,Total,2,2,2,2,2,...,2,2,2,2,2,2,2,1,1,1
93,13,Metropolitana,13104,Conchalí,Total,1,1,1,1,1,...,2,2,2,2,2,2,2,1,1,1
94,13,Metropolitana,13503,Curacaví,Rural,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1
95,13,Metropolitana,13503,Curacaví,Urbana,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
96,13,Metropolitana,13105,El Bosque,Total,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [95]:
stgo = cuarentenas[cuarentenas['codigo_comuna']==13101]
lascondes = cuarentenas[cuarentenas['codigo_comuna']==13114]

In [96]:
stgo_q = stgo.filter(regex='2021*').transpose()
lascondes_q = lascondes.filter(regex='2021*').transpose()


In [89]:
dates = [timetxttoDatetime(stgo_q.axes[0][i]) for i in range(len(stgo_q.axes[0]))]

In [90]:
dates

[datetime.datetime(2020, 7, 28, 0, 0),
 datetime.datetime(2020, 7, 29, 0, 0),
 datetime.datetime(2020, 7, 30, 0, 0),
 datetime.datetime(2020, 7, 31, 0, 0),
 datetime.datetime(2020, 8, 1, 0, 0),
 datetime.datetime(2020, 8, 2, 0, 0),
 datetime.datetime(2020, 8, 3, 0, 0),
 datetime.datetime(2020, 8, 4, 0, 0),
 datetime.datetime(2020, 8, 5, 0, 0),
 datetime.datetime(2020, 8, 6, 0, 0),
 datetime.datetime(2020, 8, 7, 0, 0),
 datetime.datetime(2020, 8, 8, 0, 0),
 datetime.datetime(2020, 8, 9, 0, 0),
 datetime.datetime(2020, 8, 10, 0, 0),
 datetime.datetime(2020, 8, 11, 0, 0),
 datetime.datetime(2020, 8, 12, 0, 0),
 datetime.datetime(2020, 8, 13, 0, 0),
 datetime.datetime(2020, 8, 14, 0, 0),
 datetime.datetime(2020, 8, 15, 0, 0),
 datetime.datetime(2020, 8, 16, 0, 0),
 datetime.datetime(2020, 8, 17, 0, 0),
 datetime.datetime(2020, 8, 18, 0, 0),
 datetime.datetime(2020, 8, 19, 0, 0),
 datetime.datetime(2020, 8, 20, 0, 0),
 datetime.datetime(2020, 8, 21, 0, 0),
 datetime.datetime(2020, 8, 22, 0,

In [99]:
fig, axs = plt.subplots(1, 1)
axs.plot(dates,stgo_q,label='stgo')
axs.plot(dates,lascondes_q,label='LasCondes')
axs.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
axs.legend(loc=0)

# Casos nuevos diarios informe diario

In [143]:
import statsmodels.api as sm
lowess = sm.nonparametric.lowess

z = lowess(I_d,list(range(len(I_d))))
#w = lowess(y, x, frac=1./3)

In [123]:
initdate = datetime(2020,12,1)

In [276]:
w = 5
endpoint = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales.csv'
casosnuevos = pd.read_csv(endpoint)
I_d = casosnuevos[casosnuevos['Fecha']=='Casos nuevos totales'].transpose()
dates = [timetxttoDatetime(I_d.axes[0][i]) for i in range(1,len(I_d.axes[0]))]
I_d = np.array(I_d.iloc[1:][6])
index = np.where(np.array(dates) >= initdate)[0][0]
dates = dates[index:]
I_d = I_d[index:]
weekdays = [date.weekday(dates[i]) for i in range(len(dates))]
I = pd.DataFrame({'dates':dates,'I':I_d,'weekdays':weekdays})
I_week = I.loc[(I['weekdays']>2) | (I['weekdays']==0)] # Sacando martes y miércoles
I_week_ra = I_week['I'].rolling(w).mean()
I_week_lowess = lowess(I_week_ra,list(range(len(I_week_ra))))

In [278]:
plt.plot(dates,I_d,label='data raw')
plt.plot(I_week['dates'],I_week_ra,linestyle = 'dashed',label='weekdays rolling 5 data points')
plt.plot(I_week['dates'][w-1:],I_week_lowess,linestyle = 'dashed',label='weekdays - lowess')
#plt.plot(I_week['dates'],I_week['I'].rolling(10).mean(),linestyle = 'dotted',label='weekdays rolling 10 5 data points')
#plt.plot(I_week['dates'],I_week_lowess,label='Week lowess')
plt.legend(loc=0)

In [201]:
w = 7
ra_I_d = np.convolve(I_d, np.ones(w), 'valid') / w

In [210]:
pdRA_I_d = np.array(I['I'].rolling(7).mean())[7:]

In [205]:
ra_pd_I_d

Rolling [window=7,center=False,axis=0]

In [237]:
I_week_lowess = lowess(I_week['I'],list(range(len(I_week['I']))))

In [242]:
plt.plot(dates,I_d,label='data raw')
#plt.plot(dates,z,label='lowess')
#plt.plot(dates[w-1:],ra_I_d,label='rolling average')
#plt.plot(dates[7:],pdRA_I_d+700,label='rolling average pandas')
#plt.plot(dates[7:],np.array(raSam_I_d)+1000,label='rolling average Sam')
plt.plot(I_week['dates'],I_week['I'].rolling(5).mean(),linestyle = 'dashed',label='weekdays rolling 5 data points')
plt.plot(I_week['dates'],I_week['I'].rolling(10).mean(),linestyle = 'dotted',label='weekdays rolling 10 5 data points')
plt.plot(I_week['dates'],I_week_lowess,label='Week lowess')
plt.legend(loc=0)

In [187]:
type(I_d)

numpy.ndarray

In [167]:
from datetime import date

L = 0.05
M = 0.05
W = 0.1
J = 0.2
V = 0.2
S = 0.2
D = 0.2

In [176]:
weights = [0.05,0.1,0.2,0.2,0.2,0.2,0.05]*18

In [178]:
weights = weights[:-2]

In [179]:
len(weights)

124

In [175]:
len(dates)%7

5

In [169]:

weights = [2 if weekday[i]<5]

In [198]:
I

,dates,I,weekdays
0,2020-12-01,1119,1
1,2020-12-02,1035,2
2,2020-12-03,1507,3
3,2020-12-04,1729,4
4,2020-12-05,1533,5
...,...,...,...
119,2021-03-30,5394,1
120,2021-03-31,6053,2
121,2021-04-01,7830,3
122,2021-04-02,8112,4


In [219]:
def rollingmean(data,window=7):
    aux = []
    for i in range(len(data)-window):
        aux.append(np.mean(data[i:window+i]))
    return(aux)

In [220]:
raSam_I_d = rollingmean(I_d)

# Deaths deis

In [249]:
endpoint = 'getDeathsByState?state='
state = '10'

In [250]:
pd.DataFrame(request(endpoint+state).json())

,dates,confirmed,suspected
0,2020-01-04T03:00:00.000Z,0,0
1,2020-01-05T03:00:00.000Z,0,0
2,2020-01-06T03:00:00.000Z,0,0
3,2020-01-07T03:00:00.000Z,0,0
4,2020-01-08T03:00:00.000Z,0,0
...,...,...,...
448,2021-03-27T03:00:00.000Z,7,5
449,2021-03-28T03:00:00.000Z,3,1
450,2021-03-29T03:00:00.000Z,3,3
451,2021-03-30T03:00:00.000Z,4,2
